# ParFuMor (version YAML & Objets)

- Ajout d'un %store pour la gestion des numéros de kanoniks (16/04/20)

- Attention les règles qui ne changent pas le radical donnent lieu à des mauvais découpages des formes...

In [1]:
# -*- coding: utf8 -*-
import os
from os.path import expanduser
import itertools
import yaml,YamlDuplicates
from yaml.constructor import ConstructorError
import warnings
import ParFuMor as PFM
from ParFuMor import *
import pickle
from IPython.display import HTML, display
#import cellbell 

In [2]:
def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

# Gestion partagée des numéros à traiter

le block suivant permet de partager les numéros à traiter entre les différents Notebooks.
- %store -r variable lit la variable dans le stock
- %store variable stocke la variable

In [1]:
%store -r numerosKalaba typeKalaba
numerosKalaba=[5]
# numerosKalaba=[18]
print numerosKalaba
typeKalaba="Kalaba"
%store numerosKalaba 
%store typeKalaba

[5]
Stored 'numerosKalaba' (list)
Stored 'typeKalaba' (str)


In [4]:
home = expanduser("~")

if typeKalaba!="Kanonik":
    repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/"
    serie=repertoire+"23-"
    nomsKalabas=[serie+"K%d/"%num for num in numerosKalaba]
else:
    repertoire=home+"/Library/Mobile Documents/com~apple~CloudDocs/Downloads/"
    serie=repertoire+"23-Kanoniks/"
    nomsKalabas=[serie+"Kanonik-%02d-ok/"%num for num in numerosKalaba]

# Traitement

In [5]:
def getExemple(lDict):
    exKey=lDict.keys()[0]
    if isinstance(lDict[exKey],dict):
        result=getExemple(lDict[exKey])
    elif isinstance(lDict[exKey],list):
        result=lDict[exKey][0]
    return result

In [6]:
PFM.duplicateErrors=[]
for serie in nomsKalabas:
    print
    print serie

    with open(serie+"Gloses.yaml", 'r') as stream:
        gloses=yaml.safe_load(stream)
        PFM.gloses=gloses
    with open(serie+"Stems.yaml", 'r') as stream:
        try:
            stems=yaml.safe_load(stream)
            PFM.stems=stems
        except ConstructorError,msg:
            print msg
            continue
    
    lexiqueTestPrep=getExemple(stems["PREP"]).lower()
    lexiqueTestNoun=getExemple(stems["NOM"]).lower()
    lexiqueTestHyper=getExemple(stems["ADJ"]).lower()
    
    with open(serie+"Blocks.yaml", 'r') as stream:
        blocks=yaml.safe_load(stream)
        PFM.blocks=blocks
    with open(serie+"Phonology.yaml", 'r') as stream:
        phonology=yaml.safe_load(stream)
        PFM.phonology=phonology
    with open(serie+"MorphoSyntax.yaml", 'r') as stream:
        morphosyntax=yaml.safe_load(stream)
        PFM.morphosyntax=morphosyntax

        
    for cat in morphosyntax["Attributs"]:
        if sorted(morphosyntax["Attributs"][cat])!=sorted(gloses[cat].keys()):
            warnings.warn("\n"+serie+"\nLes attributs de %s ne sont pas cohérents\nMorphosyntax => %s\nGloses => %s"%(cat,", ".join(morphosyntax["Attributs"][cat]),", ".join(gloses[cat].keys())))
    
    regles=Regles()
    PFM.regles=regles
    for categorie in blocks:
        regles.addBlocs(categorie,blocks[categorie])


    paradigmes=Paradigmes()
    PFM.paradigmes=paradigmes

    hierarchieCF=HierarchieCF()
    PFM.hierarchieCF=hierarchieCF
    lexique=Lexique()
    PFM.lexique=lexique


    for cat in gloses:
    #    print cat
        attributes=[]
        if gloses[cat]:
            if set(gloses[cat].keys())==set(morphosyntax["Attributs"][cat]):
                features=morphosyntax["Attributs"][cat]
    #            print "inhérent",features
            else:
                features=gloses[cat].keys()
    #            print "contextuel",features
            for attribute in features:
                attributes.append(gloses[cat][attribute])
            nuplets=(itertools.product(*attributes))
            for nuplet in nuplets:
                proprietes=[cat]
                for element in range(len(nuplet)):
                    proprietes.append(u"%s=%s"%(features[element],nuplet[element]))
                paradigmes.addForme(cat,proprietes)

    analyserGloses(gloses)
    analyserStems(stems)
        
    with open(serie+"Hierarchie-S2.pkl", 'wb') as output:
       pickle.dump(hierarchieCF, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Lexique-S2.pkl", 'wb') as output:
       pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Regles-S2.pkl", 'wb') as output:
       pickle.dump(regles, output, pickle.HIGHEST_PROTOCOL)


    PFM.lexique.lexemes[lexiqueTestPrep]

    mot=lexiqueTestNoun
    # classesNom=PFM.lexique.formeLexeme[mot.lower()][0].split('.')[1:]
    # [classeElement for classeElement in classesNom if classeElement in gloses["NOM"]["Genre"]]

    PFM.lexique.formeLexeme[lexiqueTestHyper][0]

if PFM.duplicateErrors:
    print
    print "=======ERREURS========"
    print
    for ligne in PFM.duplicateErrors:
        print ligne



/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/23-K3/
NOM : Genre Nombre Cas
VER : Temps Pers Genre
ADJ : Genre Nombre
PRO : Genre Nombre Cas
DET : Genre Nombre Cas
head stems
head stems,NOM
head stems,NOM,A
head stems,NOM,C
head stems,NOM,B
head stems,VER
head stems,VER,VI
head stems,VER,VT
head stems,ADJ
head stems,DET


### Bilan

In [7]:
stems["NOM"]

{'A': {'N1': {'bagulo': ['excuse', 'excuses'],
   'dinag': [u'obscurit\xe9', u'obscurit\xe9s'],
   'fediT': ['hurlement', 'hurlements'],
   'fesiS': ['fille', 'filles'],
   'fidaS': ['nouvelle-N', 'nouvELLE', 'nouvELLEs'],
   'kibaN': ['milieu', 'milieux'],
   'kovuv': ['cri', 'cris'],
   'kulog': ['soeur', 'soeurs', u's\u0153ur', u's\u0153urs'],
   'nerul': ['combat', 'combats'],
   'rozan': ['douleur', 'douleurs'],
   'ruzos': ['dragon', 'dragons'],
   'tuZoN': ['village', 'villages'],
   'wawar': [u'ma\xeetre', u'ma\xeetresses', u'ma\xeetresse', u'ma\xeetres']},
  'N2': {'Nugup': ['nuit', 'nuits'],
   'Subis': ['visage', 'visages'],
   'TuvuT': ['lueur', 'lueurs'],
   'ZoTef': ['voleuse', 'voleuses'],
   'dakuD': ['viande', 'viandes'],
   'gunis': ['tornade', 'tornades'],
   'kepoT': ['crapaud', 'crapauds'],
   'kudal': ['loup', 'loups'],
   'paZar': ['cuisine', 'cuisines'],
   'pavom': [u'd\xe9g\xe2t', u'd\xe9g\xe2ts'],
   'podot': ['chasseur', 'chasseurs', 'chasseuse', 'chasseuses

In [8]:
print PFM.lexique.formeLexeme[lexiqueTestHyper][0]
# print PFM.lexique.formeLexeme[lexiqueTestNoun][0]
print PFM.lexique.formeLexeme[lexiqueTestPrep][0]

blanc.A1
de


In [9]:
paradigmes.getSigmas(["VER"])

[u'VER, Temps=PRS, Pers=3Sg, Genre=A',
 u'VER, Temps=PRS, Pers=3Sg, Genre=B',
 u'VER, Temps=PRS, Pers=3Sg, Genre=C',
 u'VER, Temps=PRS, Pers=3Du, Genre=A',
 u'VER, Temps=PRS, Pers=3Du, Genre=B',
 u'VER, Temps=PRS, Pers=3Du, Genre=C',
 u'VER, Temps=PRS, Pers=3Pl, Genre=A',
 u'VER, Temps=PRS, Pers=3Pl, Genre=B',
 u'VER, Temps=PRS, Pers=3Pl, Genre=C',
 u'VER, Temps=PST, Pers=3Sg, Genre=A',
 u'VER, Temps=PST, Pers=3Sg, Genre=B',
 u'VER, Temps=PST, Pers=3Sg, Genre=C',
 u'VER, Temps=PST, Pers=3Du, Genre=A',
 u'VER, Temps=PST, Pers=3Du, Genre=B',
 u'VER, Temps=PST, Pers=3Du, Genre=C',
 u'VER, Temps=PST, Pers=3Pl, Genre=A',
 u'VER, Temps=PST, Pers=3Pl, Genre=B',
 u'VER, Temps=PST, Pers=3Pl, Genre=C']

In [10]:
case="CF=N3"
m=re.match(ur"(.*=)(.*)","CF=N1|N2")
if m:
    altTrait=m.group(1)
    altValeurs=m.group(2).split("|")
    print altTrait
    altTraits=[altTrait+v for v in altValeurs]
    print altTraits
    if any(t == case for t in altTraits):
        print "applies"
    else:
        print "does not apply"

CF=
['CF=N1', 'CF=N2']
does not apply


In [11]:
regles.getRules("VER",'CF=V1, Temps=PRS, Pers=3PL')
# regles.getRules("PRO","Nombre=Du, Genre=C")

[]

In [12]:
cat="N"
triLex={}
for element in PFM.lexique.lexemes:
#    print element
    elementElts=element.split(".")
    nom=elementElts[0]
    if len(elementElts)==3:
        if elementElts[1].startswith(cat):
            if not elementElts[2] in triLex:
                triLex[elementElts[2]]=set()
            triLex[elementElts[2]].add((PFM.lexique.lexemes[element].stem,nom))            
triLex

{}

In [13]:
for l in PFM.lexique.lexemes:
    print l,PFM.lexique.lexemes[l].stem

coup.B.N3 poDuT
main.B.N2 bilov
dernier.A1 megil
blessure.C.N1 pozeb
gargouille.B.N3 debom
fuir.VT.V1 Zizow
trois.A2 Nuras
dragon.A.N1 ruzos
placer.VT.V2 pazom
Freja.C.N2 freZa
demande.B.N3 pudos
arbre.C.N1 dedif
combattant.C.N2 dozuT
entrer.VI.V1 rakes
retourner.VI.V2 ToSaT
sort.C.N2 sijin
viande.A.N2 dakuD
lever-N.B.N2 Sonul
pendant pena
coyote.B.N3 lanur
manoir.A.N3 bedun
survoler.VT.V2 deDok
recouvrir.VT.V1 ZodaN
table.B.N2 fabeZ
nuit.A.N2 Nugup
apporter.VT.V2 Zadit
oeil.C.N1 guret
marais.C.N1 nabop
rejoindre.VT.V2 tubed
visage.A.N2 Subis
livre.C.N3 baZapa
présenter.VT.V1 zuvud
parent.C.N1 muguj
reconnaître.VT.V2 Sinop
avec nebe
héros.C.N3 roget
légende.C.N3 nuguN
douleur.A.N1 rozan
planter.VT.V2 luzof
lance.B.N3 vonude
courir.VI.V2 zuDalu
proximité.A.N3 NefTat
envahir.VT.V1 busim
voler.VT.V1 Zazun
jaune.A1 rokim
de we
deux.A2 NireN
loup.A.N2 kudal
couteau.C.N2 puzeta
combat.A.N1 nerul
chatte.B.N1 muzoN
acheter.VT.V1 bazin
demander.VT.V1 madaf
chasser-Mov.VT.V1 gagiz
disparition.A.

# Conclusion

In [14]:
ding()